In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [ ]:
# Lista de anos de 1970 até 2023
years = list(range(1970, 2023 + 1))

In [ ]:
# URLs base para cada tipo de tabela
url_templates = [
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_02",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_03&subopcao=subopt_01",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_03&subopcao=subopt_02",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_03&subopcao=subopt_03",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_03&subopcao=subopt_04",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_04",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_05&subopcao=subopt_01",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_05&subopcao=subopt_02",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_05&subopcao=subopt_03",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_05&subopcao=subopt_04",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_05&subopcao=subopt_05",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_06&subopcao=subopt_01",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_06&subopcao=subopt_02",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_06&subopcao=subopt_03",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_06&subopcao=subopt_04"
]

In [ ]:
# Inicializando um DataFrame vazio para armazenar todos os dados
all_data = pd.DataFrame()

In [ ]:
# Função para extrair dados da tabela de um ano específico
def extract_table_data(url, year):
    response = requests.get(url)
    response.raise_for_status()
    
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', class_='tb_base tb_dados')
    
    headers = [header.text.strip() for header in table.find_all('th')]
    rows = []
    for row in table.find_all('tr')[1:]:  # Ignora o cabeçalho
        cells = row.find_all('td')
        cells = [cell.text.strip() for cell in cells]
        rows.append(cells)
    
    df = pd.DataFrame(rows, columns=headers)
    df['Ano'] = year  # Adiciona uma coluna com o ano
    return df

In [ ]:
# Extraindo dados de cada ano e concatenando ao DataFrame principal
def extract_table_all_data(url):
    all_data = pd.DataFrame()
    for year in years:
        try:
            url_year = url.format(year=year)
            year_data = extract_table_data(url_year, year)
            all_data = pd.concat([all_data, year_data], ignore_index=True)
            print(f"Dados do ano {year} extraídos com sucesso.")
        except Exception as e:
            print(f"Erro ao extrair dados do ano {year}: {e}")
    return all_data

In [ ]:
all_data = extract_table_all_data(url_templates[11])

In [ ]:
all_data

In [ ]:

# Converter colunas de quantidade e valor para numérico (tratando '-' como NaN)
all_data['Quantidade (Kg)'] = pd.to_numeric(all_data['Quantidade (Kg)'], errors='coerce')
all_data['Valor (US$)'] = pd.to_numeric(all_data['Valor (US$)'], errors='coerce')

# Pivotar a tabela para que os anos fiquem como índices
df_pivot = all_data.pivot_table(index='Ano', columns='Países', values=['Quantidade (Kg)', 'Valor (US$)'], aggfunc='sum')

# Flatten the multi-level columns
df_pivot.columns = ['_'.join(col).strip() for col in df_pivot.columns.values]
df_pivot.reset_index(inplace=True)

# Exibir o DataFrame resultante
df_pivot

In [ ]:
# Convertendo a coluna 'Quantidade (L.)' para numérica (se possível)
all_data['Quantidade (L.)'] = pd.to_numeric(all_data['Quantidade (L.)'].str.replace('.', ''), errors='coerce')

In [ ]:
# Agrupando por 'Produto' e 'Ano' e somando as quantidades
all_data_grouped = all_data.groupby(['Produto', 'Ano'])['Quantidade (L.)'].sum().reset_index()

In [ ]:
# Transformando as linhas em colunas
pivot_df = all_data_grouped.pivot(index='Ano', columns='Produto', values='Quantidade (L.)').reset_index()

In [ ]:
pivot_df

In [ ]:
import json

In [ ]:
# Filtrar o DataFrame para o ano de 2022
df_2022 = pivot_df[pivot_df['Ano'] == 2022]

# Converter para JSON
json_2022 = df_2022.to_json(orient='records')
print(json_2022)

In [ ]:
# Filtrar o DataFrame para o intervalo de anos de 2020 a 2022
df_intervalo = pivot_df[(pivot_df['Ano'] >= 2020) & (pivot_df['Ano'] <= 2022)]

# Converter para JSON
json_intervalo = df_intervalo.to_json(orient='records')
print(json_intervalo)

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_table_headers(url):
    # Faz a requisição para o URL
    response = requests.get(url)
    # Cria o objeto BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    # Encontra a tabela pela classe
    table = soup.find('table', class_='tb_base tb_dados')
    # Encontra o cabeçalho da tabela
    headers = table.find('thead').find_all('th')
    # Extrai o texto dos cabeçalhos
    header_values = [header.text.strip() for header in headers]
    return header_values

# Exemplo de uso
url = 'http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_06'
headers = get_table_headers(url)
print(headers)

# Loop para realizar operações em cada URL gerada para cada ano
for url in url_templates:
    url = url.format(year=2023)
    headers = get_table_headers(url)
    print(headers)
    

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [ ]:
# Função para extrair dados de uma tabela específica
def extract_table_data(year):
    url = f'http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_02'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    table = soup.find('table', {'class': 'tb_base tb_dados'})
    
    if not table:
        return None
    
    # Coletar todas as linhas da tabela
    rows = table.find_all('tr')
    
    data = []
    for row in rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        if cols:
            data.append(cols)
    
    df = pd.DataFrame(data, columns=['Produto', f'Quantidade (L.) {year}'])
    return df

In [ ]:
# Função para extrair o texto do elemento <p> para nomear o arquivo
def get_filename_from_page(year):
    url = f'http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_02'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    p_element = soup.find('p', {'class': 'text_center'})
    if p_element:
        filename_base = p_element.text.strip().replace(f' [{year}]', '').replace(' ', '_').replace(',', '')
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f'{filename_base}_{timestamp}.csv'
        return filename
    return None